In [12]:
# tablas piolas
from tabulate import tabulate

# Librerias de análisis
import pandas as pd

# librerias de scrapping
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [2]:
proveedores = {
    "NIT": [],
    "Nombre": [],
    "tipo_organizacion": [],
    "actividad_economica": [],
    "notario": [],
    "nit_notario": [],
    "departamento": [],
    "municipio": [],
    "direccion": [],
    "telefono": [],
    "inscripcion_sat": [],
    "estado_sat": [],
    "estado_rgae": [],
    "fecha_constitucion": [],
    "inscripcion_provisional_registro_mercantil": [],
    "inscripcion_definitiva_registro_mercantil": []
}

representantes_legales = {
    "NIT_empresa": [],
    "representante_legal": [],
    "NIT_representante": []
}

In [3]:
# ubicacion de chrome driver y la url que queremos utilizar
PATH_TO_DRIVER = 'C:/chromedriver-win32/chromedriver-win32/chromedriver.exe'
url = 'https://sociedad.info/guatemala/proveedor/'

In [4]:
if False:
    # creamos el objeto service
    service = Service(PATH_TO_DRIVER)

    # se inicializa el navegador con el servicio
    driver = webdriver.Chrome(service=service)

    # esperamos a que la ventana cargue bien
    WebDriverWait(driver, 5)

    # vamos a la página elegida
    driver.get(url)

In [27]:
# leemos el archivo con la información y los nit (que es lo que mas nos importa)
df = pd.read_excel('../data/PROV_CPE_2005-2024_NOV.xlsx')
done1 = pd.read_excel('../data/PROVEEDORES.xlsx')
done2 = pd.read_excel('../data/PROVEEDORES_parte_2.xlsx')
done3 = pd.read_excel('../data/PROVEEDORES_parte_3.xlsx')
done4 = pd.read_excel('../data/PROVEEDORES_parte_4.xlsx')

In [28]:
done = pd.concat([done1, done2, done3, done4], axis=0).reset_index()
done = done.drop_duplicates(keep='first')
done.shape

(7869, 17)

In [29]:
nits = df.NIT.unique()
nitsdone = done.NIT.unique()
len(nitsdone)

7869

In [30]:
leftnits = [nit for nit in nits if nit not in nitsdone]

In [34]:
pd.DataFrame({'NITs_faltantes': leftnits}).to_excel('../data/nits_faltantes_luego_de_parte_4.xlsx', index=False)

In [31]:
tablenit = [
    ['nits_totales', len(nits)],
    ['nits_hechos', len(nitsdone)],
    ['nits_faltantes', len(leftnits)]
]
print(tabulate(tablenit, headers=['Descripción', 'Cantidad'], tablefmt='grid'))

+----------------+------------+
| Descripción    |   Cantidad |
+================+============+
| nits_totales   |       7920 |
+----------------+------------+
| nits_hechos    |       7869 |
+----------------+------------+
| nits_faltantes |         51 |
+----------------+------------+


In [32]:
def extraer_campo(soup, class_name : str) -> str:
    '''para a valores comprendidos en dos etiquetas span'''
    field_span = soup.find('span', class_=class_name)
    if field_span:
        value_span = field_span.find('span', class_='fieldValue')
        return value_span.get_text(strip=True) if value_span else None
    return None


def extraer_representantes_legales(soup) -> list[tuple[str,str]]:
    '''Extrae los NIT y nombres de los representantes legales'''
    info_rl = []
    
    # Buscar el contenedor principal de los resultados
    results_div = soup.find('div', class_='results')

    # Recorrer todos los resultados dentro del contenedor
    for result in results_div.find_all('a', class_='result'):
        # Buscar el nombre y el NIT en sus respectivas etiquetas
        name_tag = result.find('div', class_='resultName').find('span')
        nit_tag = result.find('span', class_='field rep_nit').find_next('span', class_='fieldValue')

        # Si ambos elementos son encontrados, agregar la tupla de NIT y nombre a la lista
        if name_tag and nit_tag:
            info_rl.append((nit_tag.get_text(strip=True), name_tag.get_text(strip=True)))
    
    return info_rl

In [ ]:
if False:
    for nit in nits:
        print(f'INICIO {nit}...')

        # realizamos la solicitud
        print('REQUEST...')
        response = requests.get('https://sociedad.info/guatemala/proveedor/' + nit, proxies={'http': None, 'https': None}, timeout=60)

        # parseamos el texto a html
        print('PARSE...')
        soup = BeautifulSoup(response.content, 'html.parser')

        # lo guardamos para aprovecharlo después
        print('GUARDANDO...')
        with open('../data/html_proveedores/' + nit + '.html', 'w') as si: 
            si.write(soup.prettify())

        # FIN
        print('FIN...')

        # esperamos 1 segundo por si acaso.
        time.sleep(1)


In [33]:
i = 0
nit_failed = []
for nit in leftnits:
    try:
        print(f'INICIO {nit}...')
        # realizamos la solicitud
        print('REQUEST...')
        response = requests.get('https://sociedad.info/guatemala/proveedor/' + nit, proxies={'http': None, 'https': None}, timeout=60)
        

        # parseamos el texto a html
        print('PARSE...')
        soup = BeautifulSoup(response.content, 'html.parser')
        

        print('GUARDANDO...')
        with open('../data/html_proveedores/' + nit + '.html', 'w') as si: 
            si.write(soup.prettify())
        

        # Creamos un diccionario temporal para la información del proveedor
        proveedor = {
            'NIT': nit,
            'Nombre': soup.find('span', class_='itemName').text.strip(),
            'tipo_organizacion': extraer_campo(soup, 'field tipo_organizacion'),
            'actividad_economica': extraer_campo(soup, 'field actividad_economica field_main'),
            'notario': extraer_campo(soup, 'field notario_nombre field_main'),
            'nit_notario': extraer_campo(soup, 'field notario_nit field_main'),
            'departamento': extraer_campo(soup, 'field departamento'),
            'municipio': extraer_campo(soup, 'field municipio'),
            'direccion': extraer_campo(soup, 'field direccion field_main'),
            'telefono': extraer_campo(soup, 'field telefono field_main'),
            'inscripcion_sat': extraer_campo(soup, 'field fecha_sat field_main'),
            'estado_sat': extraer_campo(soup, 'field estatus_nit'),
            'estado_rgae': extraer_campo(soup, 'field estado_proveedor_rgae field_main'),
            'fecha_constitucion': extraer_campo(soup, 'field fecha_constitucion field_main'),
            'inscripcion_provisional_registro_mercantil': extraer_campo(soup, 'field fecha_provisional_registro_mercantil field_main'),
            'inscripcion_definitiva_registro_mercantil': extraer_campo(soup, 'field fecha_definitiva_registro_mercantil field_main')
        }
        print('INFORMACION EXTRAIDA...')

        # Agregamos los datos del proveedor al diccionario principal
        for key, value in proveedor.items():
            proveedores[key].append(value)


        # representantes legales
        representantes = extraer_representantes_legales(soup)
        for repnit, nombre in representantes:
            representantes_legales['NIT_empresa'].append(nit)
            representantes_legales['representante_legal'].append(nombre)
            representantes_legales['NIT_representante'].append(repnit)
        
        i += 1
        print(i)
        print(f'FIN {nit}...')
    except Exception as e:
        nit_failed.append(nit)
        print(f'FAILED {nit}...')

        
proveedores_df = pd.DataFrame(proveedores)
representantes_df = pd.DataFrame(representantes_legales)

proveedores_df.to_excel('../data/PROVEEDORES_parte_4.xlsx', index=False)
representantes_df.to_excel('../data/REPRESENTANTES_LEGALES_parte_4.xlsx', index=False)


proveedores_df.to_csv('../data/PROVEEDORES_parte_4.csv', sep=',', index=False)
representantes_df.to_csv('../data/REPRESENTANTES_LEGALES_parte_4.csv', sep=',', index=False)

INICIO 9899553...
REQUEST...
PARSE...
GUARDANDO...
INFORMACION EXTRAIDA...
FAILED 9899553...
INICIO 118193627...
REQUEST...
PARSE...
GUARDANDO...
FAILED 118193627...
INICIO 115462821...
REQUEST...
PARSE...
GUARDANDO...
FAILED 115462821...
INICIO 115055525...
REQUEST...
FAILED 115055525...
INICIO 108735931...
REQUEST...
PARSE...
GUARDANDO...
FAILED 108735931...
INICIO 117419974...
REQUEST...
PARSE...
GUARDANDO...
FAILED 117419974...
INICIO 120142988...
REQUEST...
PARSE...
GUARDANDO...
FAILED 120142988...
INICIO 117993166...
REQUEST...
PARSE...
GUARDANDO...
FAILED 117993166...
INICIO 108979415...
REQUEST...
PARSE...
GUARDANDO...
FAILED 108979415...
INICIO 101953496...
REQUEST...
PARSE...
GUARDANDO...
FAILED 101953496...
INICIO 120217368...
REQUEST...
FAILED 120217368...
INICIO 117760617...
REQUEST...
FAILED 117760617...
INICIO 120068907...
REQUEST...
PARSE...
GUARDANDO...
FAILED 120068907...
INICIO 119703866...
REQUEST...
FAILED 119703866...
INICIO 120003376...
REQUEST...
PARSE...
GUARDA

In [ ]:
if False:
    # Nombre empresa o proveedor
    name = soup.find('span', class_='itemName').text.strip()

    # extraer NIT
    nit = soup.find('span', class_='field nit_propio').find_next('span', class_='fieldValue').text.strip()

    # extraemos tipo de organización
    tipo_organizacion = soup.find('span', class_='field tipo_organizacion').find_next('span', class_='fieldValue').text.strip()

    # actividad economica
    actividad = soup.find('span', class_='field actividad_economica field_main').find_next('span', class_='fieldValue').text.strip()

    # Nombre del notario
    notario = soup.find('span', class_='field notario_nombre field_main').find_next('span', class_='fieldValue').text.strip()

    # nit del notario
    nit_notario = soup.find('span', class_='field notario_nit field_main').find_next('span', class_='fieldValue').text.strip()

    # departamento
    departamento = soup.find('span', class_='field departamento').find_next('span', class_='fieldValue').text.strip()

    # municipio
    municipio = soup.find('span', class_='field municipio').find_next('span', class_='fieldValue').text.strip()

    # direccion
    direccion = soup.find('span', class_='field direccion field_main').find_next('span', class_='fieldValue').text.strip()

    # telefono
    telefono = soup.find('span', class_='field telefono field_main').find_next('span', class_='fieldValue').text.strip()

    insc_sat = extraer_campo(soup, 'field fecha_sat field_main')
    status_sat = extraer_campo(soup, 'field estatus_nit')
    rgae_status_proveedor1 = extraer_campo(soup, 'field estado_proveedor_rgae field_main')
    fecha_constitucion = extraer_campo(soup, 'field fecha_constitucion field_main')
    insc_prov_reg_merc = extraer_campo(soup, 'field fecha_provisional_registro_mercantil field_main')
    insc_defin_reg_merc = extraer_campo(soup, 'field fecha_definitiva_registro_mercantil field_main')


    # informacion representantes legales
    info_rl = []

    results_div = soup.find('div', class_='results')


    for result in results_div.find_all('a', class_='result'):
        name_tag = result.find('div', class_='resultName').find('span')
        nit_tag = result.find('span', class_='field rep_nit').find_next('span', class_='fieldValue')

        if name_tag and nit_tag:
            info_rl.append((nit_tag.get_text(strip=True), name_tag.get_text(strip=True)))